In [2]:
#criteria: at least 5 games played in regular season before or after event
#data partially from http://nbafamily.wikia.com/wiki/Category:NBA_Children

player_events=[['J.R. Smith','02/09/2009','birth'], \
               ['Paul Pierce','04/04/2008','birth'], \
               ['Paul Pierce','04/02/2016','birth'], \
               ['Dwyane Wade','02/04/2002','birth'], \
               ['Dwyane Wade','11/10/2013','birth'], \
               ["Shaquille O'Neal",'01/11/2000','birth'], \
               ["Shaquille O'Neal",'11/13/2001','birth'], \
               ["Carmelo Anthony",'03/07/2007','birth'], \
               ["Chris Bosh",'11/04/2013','birth'], \
               ["Kobe Bryant",'01/19/2003','birth'], \
               ["Michael Jordan",'12/07/1992','birth'], \
               ["Michael Jordan",'11/18/1988','birth'], \
               ["Michael Jordan",'12/24/1990','birth'], \
               ['Steve Nash','11/12/2010','birth'], \
               ['Steve Nash','11/12/2010','divorce'], \
               ['Tim Duncan','03/27/2013','divorce'], \
               ["Amar'e Stoudemire",'02/06/2012','death'], \
               ['Mychal Thompson','02/08/1990','child'], \
               ['Tony Parker','11/17/2010','divorce']]

In [36]:
#input player, event date

import requests,bs4
import pandas as pd
import datetime
import numpy as np

# using "Historical" sheet from Box Plus Minus 1.1 Historical for BBRef.xlsm 
# from https://www.basketball-reference.com/about/bpm.html

xl = pd.ExcelFile("BPM_1_1_Historical_BBRef.xls")
bbrefdf = xl.parse("Sheet1")

#get input of player name and life event date as MM/DD/YYYY
for p_ind,player_info in enumerate(player_events):
    player_tofind=player_info[0]
    date_lifeevent=pd.to_datetime(player_info[1])
    print player_info
    print [player_tofind, date_lifeevent]   

# # 
# # #get player URL
# # first_letter=player_tofind[player_tofind.find(' ')+1].lower()
# # url='https://www.basketball-reference.com/players/'+first_letter+'/'
# # 
# # #wrap in url2soup?
# # res=requests.get(url)
# # bbsoup=bs4.BeautifulSoup(res.text,'lxml')
# # body=bbsoup.select('tbody')
# # 
# # names=body[0].find_all('tr')
# 
# for player_index,player in enumerate(names):
#     if player_tofind==str(player.find_all('a')[0].contents[0]):
#         urlnew='https://www.basketball-reference.com/'+player.find_all('a')[0].attrs['href']
#         break
#     
    #get season URL
    date_month=date_lifeevent.month
    if date_month > 9:
        year_tofind=date_lifeevent.year+1
    else:
        year_tofind=date_lifeevent.year    
    
    playerslice=bbrefdf[(bbrefdf['Player']==player_tofind) & (bbrefdf['Year']==year_tofind)]
    playertarget_id=playerslice['Player ID']
    playertarget_id = str(playertarget_id.values[0])
    if len(playerslice)==0:
        continue
    player_bpm = float(playerslice['BPM'])
    minBPM = player_bpm*0.8
    maxBPM = player_bpm*1.2
    
    print playertarget_id
    # 
    # player_name = pl_seas[1][1]
    # season = pl_seas[1][0]
    # if not((player_name == player_tofind) & (season == year_tofind)):
    #     player_id=pl_seas[1][2]
    #     url = 'https://www.basketball-reference.com/players/'+str(player_id[0])+'/'+str(player_id)+'/gamelog/'+str(season)+'/'
    #     
    #get GameScores for specific date range
    gamescores_preevent=pd.DataFrame()
    #between start date and event date
    url = 'https://www.basketball-reference.com/players/'+playertarget_id[0]+'/'+playertarget_id+'/gamelog/'+str(year_tofind)+'/'
    date_start = date_lifeevent-datetime.timedelta(days=31)
    date_end = date_lifeevent
    playerdf = get_gamescores(url, date_start, date_end)
    pid = pd.Series.repeat(pd.Series(playertarget_id), len(playerdf))
    playerdf['Player ID'] = pid.values
    print [player_tofind, 'mean/SD of GameScore:', np.mean(playerdf['GameScore']), np.std(playerdf['GameScore'])]
    gamescores_preevent = gamescores_preevent.append(playerdf)
            
    gamescores_postevent = pd.DataFrame()
    #between event date and end date
    url = 'https://www.basketball-reference.com/players/'+playertarget_id[0]+'/'+playertarget_id+'/gamelog/'+str(year_tofind)+'/'
    date_start = date_lifeevent
    date_end = date_lifeevent+datetime.timedelta(days=31)
    playerdf = get_gamescores(url, date_start, date_end)
    pid = pd.Series.repeat(pd.Series(playertarget_id), len(playerdf))
    playerdf['Player ID'] = pid.values
    print [player_tofind, 'mean/SD of GameScore:', np.mean(playerdf['GameScore']), np.std(playerdf['GameScore'])]
    gamescores_postevent = gamescores_postevent.append(playerdf)
    
    # get season BPM for player

    # find similar player-seasons within 10% of player's season BPM
    
    # record distribution of GameScores for those player-seasons
    
    samebpm = bbrefdf[(bbrefdf['BPM'] > minBPM) & (bbrefdf['BPM'] < maxBPM)]
    samebpm_data = samebpm[['Year', 'Player', 'Player ID', 'BPM', 'G']]

    #floor of 15 games that season to compare appropriate sample sizes
    samebpm_data = samebpm_data[samebpm_data['G'] > 15]
    gamescores_all = pd.DataFrame()
    for pl_seas in samebpm_data.iterrows():
        player_name = pl_seas[1][1]
        season = pl_seas[1][0]
        
        if (player_name == player_tofind) & (season == year_tofind):
            continue
        
        if season<1984: #steals, blocks not reliably tracked until then
            continue
        
        player_id = pl_seas[1][2]
        date_start = pd.to_datetime('09/01/'+str(season-1))
        date_end = pd.to_datetime('06/01/'+str(season))            
        url = 'https://www.basketball-reference.com/players/'+str(player_id[0])+'/'+str(player_id)+'/gamelog/'+str(season)+'/'
        playerdf = get_gamescores(url, date_start, date_end)
        pid = pd.Series.repeat(pd.Series(str(player_id)), len(playerdf))
        playerdf['Player ID']=pid.values
        print [str(player_name), 'mean/SD of GameScore:', np.mean(playerdf['GameScore']), np.std(playerdf['GameScore'])]
        gamescores_all = gamescores_all.append(playerdf)
            
    print ['Overall mean/SD: ', np.mean(gamescores_all['GameScore']), np.std(gamescores_all['GameScore'])]


['J.R. Smith', '02/09/2009', 'birth']
['J.R. Smith', Timestamp('2009-02-09 00:00:00')]
smithjr01


['J.R. Smith', 'mean/SD of GameScore:', 9.6076923076923073, 7.7468069390865351]


['J.R. Smith', 'mean/SD of GameScore:', 8.9600000000000009, 5.6115119768799095]


['Rolando Blackman', 'mean/SD of GameScore:', 16.91604938271605, 7.4354083931462691]


['Buck Williams', 'mean/SD of GameScore:', 13.481249999999999, 6.2942254040906418]


...

['Bill Laimbeer', 'mean/SD of GameScore:', 14.937804878048778, 6.9533224247416587]


['Mark Eaton', 'mean/SD of GameScore:', 10.780487804878049, 5.6370819086627817]


['Derek Smith', 'mean/SD of GameScore:', 15.175000000000001, 6.9326311743810516]


['Mark Aguirre', 'mean/SD of GameScore:', 16.708750000000002, 8.2070578429483483]


KeyboardInterrupt: 

In [34]:
season<1978

False

In [ ]:

# is month before life-event (MBLE) different from player-season distribution

# is MPLE different from month after life event (MALE)?


In [9]:
type(player_name)

NameError: name 'player_name' is not defined

In [7]:
def get_gamescores(url,date_start,date_end):
    #get GameScores from given Basketball Reference URL
    
    import requests, bs4
    import pandas as pd
    from datetime import timedelta

    
    dftemp = []
    res = requests.get(url)
    bbsoup = bs4.BeautifulSoup(res.text,'lxml')
    body = bbsoup.select('tbody')
    
    games = body[0].find_all('tr')
    for game_index, game in enumerate(games):
        stats = game.find_all('td')
        for stat_index, stat in enumerate(stats):
            # check if GameScore exists as a stat, and that the player played that game
            gamedate=pd.to_datetime(stats[1].getText())
            if (gamedate>date_start) & (gamedate<date_end):
                if (stat.attrs['data-stat'] == 'game_score') & (len(stat) > 0):
                    dftemp.append([str(stats[1].contents[0].contents[0]), float(stats[stat_index].contents[0])])
    df = pd.DataFrame(dftemp, columns=['Date', 'GameScore'])
    return df